# Data Understanding

Proses *Data Understanding* merupakan tahap awal dalam siklus analisis data yang berfokus pada pengumpulan, penelaahan, dan evaluasi dataset. Tujuan utamanya adalah memastikan bahwa data yang digunakan benar-benar relevan dengan permasalahan bisnis serta layak diproses lebih lanjut dalam tahap analisis.

Tahapan utama dalam memahami data meliputi:

### 1. Pengumpulan Data  
Mengidentifikasi serta memperoleh data dari berbagai sumber yang berhubungan dengan permasalahan yang sedang dikaji.

### 2. Deskripsi Data  
Menganalisis struktur data, seperti jumlah baris dan kolom, serta memahami tipe variabel atau atribut yang tersedia.

### 3. Eksplorasi Data  
Melakukan penelusuran awal untuk menemukan pola, hubungan, maupun penyimpangan dalam data. Tahapan ini biasanya menggunakan metode statistik deskriptif dan visualisasi data.

### 4. Kualitas Data  
Menilai kelengkapan, akurasi, dan konsistensi data untuk memastikan dataset memiliki mutu yang memadai sebelum dianalisis lebih jauh.


In [9]:
%pip install seaborn
# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import PyCaret untuk EDA dan classification
try:
    from pycaret.datasets import get_data
    from pycaret.classification import *
    from pycaret.anomaly import *
    print("PyCaret berhasil diimport")
except ImportError:
    print("PyCaret tidak tersedia. Install dengan: pip install pycaret")

# visualisasi
plt.style.use('default')
sns.set_palette("husl")

print("Library berhasil diimport")

Note: you may need to restart the kernel to use updated packages.
PyCaret berhasil diimport
Library berhasil diimport



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load Dataset

Dataset Iris akan dimuat dari file CSV. Pastikan file CSV berada di folder yang sama atau sesuaikan path-nya.

In [10]:
# Load dataset Iris dari file CSV atau PyCaret
try:
    df = pd.read_csv('data_iris.csv', delimiter=';')

    numeric_columns = ['sepal length', 'sepal width', 'petal length', 'petal width']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

    if 'Class' in df.columns:
        df['species_name'] = df['Class'].map({
            'Iris-setosa': 'setosa',
            'Iris-versicolor': 'versicolor',
            'Iris-virginica': 'virginica'
        })
        df['species'] = df['species_name'].map({
            'setosa': 0, 'versicolor': 1, 'virginica': 2
        })

    df = df.rename(columns={
        'sepal length': 'sepal length (cm)',
        'sepal width': 'sepal width (cm)',
        'petal length': 'petal length (cm)',
        'petal width': 'petal width (cm)'
    })

    if 'id' in df.columns:
        df = df.drop('id', axis=1)

    print("✅ Dataset Iris berhasil dimuat dari data_iris.csv")

except FileNotFoundError:
    print("⚠️ File lokal tidak ditemukan. Menggunakan dataset Iris dari PyCaret...")
    from pycaret.datasets import get_data
    try:
        df = get_data('iris')

        df['species_num'] = df['species'].map({
            'setosa': 0, 'versicolor': 1, 'virginica': 2
        })
        df['species_name'] = df['species']

        print("✅ Dataset Iris berhasil dimuat dari PyCaret")
    except Exception as e:
        print("❌ Error: Tidak dapat memuat dataset dari PyCaret")
        print(e)

print(f"\nUkuran dataset: {df.shape[0]} baris, {df.shape[1]} kolom")
print(f"Kolom dataset: {list(df.columns)}")

df.head()

✅ Dataset Iris berhasil dimuat dari data_iris.csv

Ukuran dataset: 150 baris, 1 kolom
Kolom dataset: ['id,Class,sepal length,sepal width,petal length,petal width']


,"id,Class,sepal length,sepal width,petal length,petal width"
0,"1,Iris-setosa,5.1,3.5,1.4,0.2"
1,"2,Iris-setosa,4.9,3,1.4,0.2"
2,"3,Iris-setosa,4.7,3.2,1.3,0.2"
3,"4,Iris-setosa,4.6,3.1,1.5,0.2"
4,"5,Iris-setosa,5,3.6,1.4,0.2"


## Informasi Dataset

Mengecek tipe data tiap kolom, jumlah baris, dan ada tidaknya missing values.

In [11]:
import pandas as pd
from sklearn.datasets import load_iris
from IPython.display import display

IRIS = load_iris()

columns_clean = [col.replace(" (cm)", "") for col in IRIS.feature_names]
df = pd.DataFrame(data=IRIS.data, columns=columns_clean)

df['species'] = IRIS.target + 1

df['species_name'] = df['species'].map({1: 'setosa', 2: 'versicolor', 3: 'virginica'})

df.insert(0, 'id', range(1, len(df)+1))

print("Dataset Iris berhasil dimuat")
print(f"Ukuran dataset: {df.shape[0]} baris, {df.shape[1]} kolom")

pd.options.display.float_format = "{:.1f}".format

display(
    df.head().style.format("{:.1f}", subset=df.columns[1:5]).hide(axis="index")
)

Dataset Iris berhasil dimuat
Ukuran dataset: 150 baris, 7 kolom


id,sepal length,sepal width,petal length,petal width,species,species_name
1,5.1,3.5,1.4,0.2,1,setosa
2,4.9,3.0,1.4,0.2,1,setosa
3,4.7,3.2,1.3,0.2,1,setosa
4,4.6,3.1,1.5,0.2,1,setosa
5,5.0,3.6,1.4,0.2,1,setosa


## Mengecek Tipe Data dan Jenis Variabel


In [12]:
print("\n--- Tipe Data Kolom ---")
print(df.dtypes)

print("\n--- Mengecek Missing Value ---")
print(df.isnull().sum())


--- Tipe Data Kolom ---
id                int64
sepal length    float64
sepal width     float64
petal length    float64
petal width     float64
species           int32
species_name     object
dtype: object

--- Mengecek Missing Value ---
id              0
sepal length    0
sepal width     0
petal length    0
petal width     0
species         0
species_name    0
dtype: int64


## Statistik Deskriptif Data

Menampilkan statistik dasar dari kolom numerik, seperti min, max, mean, dan standar deviasi.



In [13]:
df.describe()

,id,sepal length,sepal width,petal length,petal width,species
count,150.0,150.0,150.0,150.0,150.0,150.0
mean,75.5,5.8,3.1,3.8,1.2,2.0
std,43.4,0.8,0.4,1.8,0.8,0.8
min,1.0,4.3,2.0,1.0,0.1,1.0
25%,38.2,5.1,2.8,1.6,0.3,1.0
50%,75.5,5.8,3.0,4.3,1.3,2.0
75%,112.8,6.4,3.3,5.1,1.8,3.0
max,150.0,7.9,4.4,6.9,2.5,3.0


## Statistik Kategori Spesies

Menampilkan jumlah masing-masing spesies iris untuk mengetahui distribusi kelas.

In [14]:
df['species'].value_counts()

species
1    50
2    50
3    50
Name: count, dtype: int64